In [1]:
import csv
import os,re,csv
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim import corpora, models
import gensim
import numpy as np
import scipy
import math
import matplotlib.pyplot as plt
import plotly.plotly as py
import twitter
import json
import pandas as pd

In [2]:
import tweepy
import csv
import json

In [3]:
df1 = pd.read_csv('personalities.csv')

In [4]:
df2 = pd.read_csv('personalities_next_60.csv')

In [5]:
df3 = pd.read_csv('personalities_last.csv')

In [13]:
df1[0:62]

,Personality,username,user_id,tweets,followers_count,image
0,enfj,pr3achlikeagirl,2.310891e+09,"['God is on the move!', 'Stop telling God what...",67,http://pbs.twimg.com/profile_images/1090079036...
1,enfj,ChurchTherapist,8.606921e+08,['@user It describes anxious attachment as opp...,1841,http://pbs.twimg.com/profile_images/1033441323...
2,enfj,camperry21,2.653982e+07,"[""@user @user He ain't winning with the Clippe...",1240,http://pbs.twimg.com/profile_images/1078547897...
3,enfj,galaxiaskykloz,2.268542e+09,"['does anyone remember poopreport dot com', '@...",2227,http://pbs.twimg.com/profile_images/1001856767...
4,enfj,sarahyoung_esq,1.090463e+18,"['MeatPotatoesNBeans ', 'THE LITTLE BLEP TONGU...",76,http://pbs.twimg.com/profile_images/1090465009...
5,enfj,filmandtheatre,3.686383e+08,['oh my god she is adorable i love her so much...,6031,http://pbs.twimg.com/profile_images/1137455550...
6,enfj,MeghanLeVota,1.016060e+18,['@user I’m an ENFJ 1) individualization 2) fu...,1038,http://pbs.twimg.com/profile_images/1133759007...
7,enfp,ukthxbyek,1.069713e+18,['@user I realize I lied because if I file the...,225,http://pbs.twimg.com/profile_images/1135702016...
8,enfp,2dancingpandas,1.064558e+18,['@user Wow! What a great picture! Expresses...,310,http://pbs.twimg.com/profile_images/1111719068...
9,enfp,DotsWritingDojo,8.926282e+17,['@user Good that you shared this online. Very...,253,http://pbs.twimg.com/profile_images/1136308065...


In [8]:
df1.shape

(175, 6)

In [14]:
df1 = df1[0:62]

In [15]:
df = pd.concat([df1, df2, df3], ignore_index = True)

In [16]:
df

,Personality,username,user_id,tweets,followers_count,image
0,enfj,pr3achlikeagirl,2.310891e+09,"['God is on the move!', 'Stop telling God what...",67,http://pbs.twimg.com/profile_images/1090079036...
1,enfj,ChurchTherapist,8.606921e+08,['@user It describes anxious attachment as opp...,1841,http://pbs.twimg.com/profile_images/1033441323...
2,enfj,camperry21,2.653982e+07,"[""@user @user He ain't winning with the Clippe...",1240,http://pbs.twimg.com/profile_images/1078547897...
3,enfj,galaxiaskykloz,2.268542e+09,"['does anyone remember poopreport dot com', '@...",2227,http://pbs.twimg.com/profile_images/1001856767...
4,enfj,sarahyoung_esq,1.090463e+18,"['MeatPotatoesNBeans ', 'THE LITTLE BLEP TONGU...",76,http://pbs.twimg.com/profile_images/1090465009...
5,enfj,filmandtheatre,3.686383e+08,['oh my god she is adorable i love her so much...,6031,http://pbs.twimg.com/profile_images/1137455550...
6,enfj,MeghanLeVota,1.016060e+18,['@user I’m an ENFJ 1) individualization 2) fu...,1038,http://pbs.twimg.com/profile_images/1133759007...
7,enfp,ukthxbyek,1.069713e+18,['@user I realize I lied because if I file the...,225,http://pbs.twimg.com/profile_images/1135702016...
8,enfp,2dancingpandas,1.064558e+18,['@user Wow! What a great picture! Expresses...,310,http://pbs.twimg.com/profile_images/1111719068...
9,enfp,DotsWritingDojo,8.926282e+17,['@user Good that you shared this online. Very...,253,http://pbs.twimg.com/profile_images/1136308065...


In [17]:
personalities_FINAL = df.to_csv ('personalities_FINAL.csv', index = None, header=True)